<a href="https://colab.research.google.com/github/lauberto/politopic/blob/main/notebook/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modelling on Politicians' speech

This notebook shows how to apply topic modelling on the transcription of yt videos with Politicians' speech. The transcription is performed with [`whisper`](https://github.com/openai/whisper).

## Installing dependencies

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt update && sudo apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,567 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,095 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease

In [2]:
# Installing whisper
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8e332mz2
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-8e332mz2
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 182 kB 86.2 MB/s 
     |████████████████████████████████| 7.6 MB 50.0 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175327 sha256=dae2e7abe1ca055adab0f26ad0492261ba4ffc11b9583d3906e74af564446bbe
  Stored in directory: /tmp/pip-ephem-wheel-cache-ovay7ntn/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


 Installing [`pytube`](https://https://github.com/pytube/pytube). and downloading the [video](https://https://www.youtube.com/watch?v=knj8ULToNvo) from youtube.

In [3]:
!pip install git+https://github.com/pytube/pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytube/pytube to /tmp/pip-req-build-my32wrxi
  Running command git clone -q https://github.com/pytube/pytube /tmp/pip-req-build-my32wrxi
  Created wheel for pytube: filename=pytube-12.1.0-py3-none-any.whl size=56809 sha256=13bbbb2c2a49414c1542945304ada5f564fcecac79fff5bbed0d623e3c951ee0
  Stored in directory: /tmp/pip-ephem-wheel-cache-03e_1zep/wheels/a8/ac/8c/337af6a10cc543c5eadf4eb2bbd02bd8609b25bea729df338a
Successfully built pytube


## First Demo - Salvini's Speech at a Coldiretti Conference.

Who's better than a populist far-right politician like Salvini to start with the analysis of political speeches?

In [4]:
video_ids = ['knj8ULToNvo', 'wVHyFwyhQwE&t=73s', 'y1qlZ077zJI&t=162s', '4OQmieY4a-Q', 'V9w4ZtZisrs']
video_urls = ['https://youtu.be/' + video_id for video_id in video_ids]

### Download & Transcription
Download the video from YouTube and transcribe it using whisper. We are going to 

In [5]:
import whisper
from pytube import YouTube

In [7]:
whisper_model = whisper.load_model('base')

100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 29.0MiB/s]


In [14]:
titles = []
texts = []

for url in video_urls:
  yt = YouTube(url=url)
  titles.append(yt.title)
  path = yt.streams.filter(only_audio=True)[0].download(filename="audio.mp4")
  transcription = whisper_model.transcribe(path)
  texts.append(transcription["text"])  

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


### Sentence segmentation

In [29]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
from nltk import sent_tokenize

In [31]:
docs = []
video_titles = []

for title, text in zip(titles, texts):
  sents = sent_tokenize(text)
  docs.extend(sents)
  video_titles.extend([title]*len(sents))

Let's save the texts on google drive just in case.

In [33]:
import pandas as pd

df = pd.DataFrame({"Title": video_titles, "Text": docs})
df.head()

,Title,Text
0,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),accogliamo un ultimo dei nostri amici.
1,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Parlaramo poco fammi, ha detto, sono una perso..."
2,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),Quindi diamo la parola per ultimo proprio il s...
3,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Cioè, presentazione alla Gesmundo New Look, se..."
4,ASSEMBLEA NAZIONALE COLDIRETTI (28.07.2022),"Non ho il compitino pronto, cioè il mio uffici..."


In [41]:
df.to_csv('/content/drive/MyDrive/Data/political_speeches/politopic_salvini.tsv', sep='\t', index=False)

In [8]:
import pandas as pd

In [10]:
df = pd.read_csv('/content/drive/MyDrive/Data/political_speeches/politopic_salvini.tsv', sep='\t')

## Latent Dirichlet Allocation

In [14]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
it_stopwords = stopwords.words('italian')

In [66]:
it_stopwords[:5]

['ad', 'al', 'allo', 'ai', 'agli']

In [7]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(max_df=0.9, min_df=3, stop_words=it_stopwords)
lda = LatentDirichletAllocation(n_components=5)

In [64]:
count = cv.fit_transform(df.Text)
lda.fit(count)

LatentDirichletAllocation(n_components=5)

count.toArray()

In [65]:
cv.get_feature_names()[:5]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['agricoltori', 'agricoltura', 'alimentare', 'assolutamente', 'centro']

In [60]:
def get_feature_names_per_topic(lda_components, count_vectorizer, topn=5):
  topics = []

  for i in range(len(lda_components)):
    topic = lda_components[i]
    top_words_idx = topic.argsort()[-topn:]
    topic_names = [count_vectorizer.get_feature_names_out()[id] for id in top_words_idx]
    topics.append('_'.join(topic_names))
  
  return topics

In [62]:
topics = get_feature_names_per_topic(lda.components_, cv)

In [63]:
topics

['però_quindi_italiani_senza_inno',
 'quindi_poi_devo_ringrazio_fatto',
 'prendo_grazie_soldi_costa_fa',
 'intusiasta_cioè_problema_sì_alimentare',
 'salute_ultimo_agricoltura_senza_quando']